In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os 
import pandas as pd
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
import requests
import json

# variables
browser = webdriver.Chrome()
firstLogin = 'insert email'
firstPassword = 'insert password'

# open browser
browser.implicitly_wait(3) # seconds
browser.get("https://www.livoltek-portal.com/#/")
browser.maximize_window()

# login Livoltek account
browser.find_element_by_xpath('//*[@id="app"]/div/div/div/form/div[2]/div/div[1]/input').send_keys(firstLogin)
browser.find_element_by_xpath('//*[@id="app"]/div/div/div/form/div[3]/div/div[1]/input').send_keys(firstPassword)
browser.find_element_by_xpath('//*[@id="app"]/div/div/div/button').click()
time.sleep(2)

totalData=pd.DataFrame()
dataResult=pd.DataFrame()

# get table
RTP=browser.find_elements_by_xpath('//*[@id="app"]/section/main/div/div[4]/div/div/div[3]/div[2]/table/tbody/tr/td[1]/div/div/div/p[1]/span[2][1]')
siteStatus=browser.find_elements_by_xpath('//*[@id="app"]/section/main/div/div[4]/div/div/div[3]/div[2]/table/tbody/tr/td[1]/div/div/div/p[2]/span[2]/div[1]')
PVC=browser.find_elements_by_xpath('//*[@id="app"]/section/main/div/div[4]/div/div/div[3]/div[2]/table/tbody/tr/td[1]/div/div/div/p[3]/span[2][1]')
SCT=browser.find_elements_by_xpath('//*[@id="app"]/section/main/div/div[4]/div/div/div[3]/div[2]/table/tbody/tr/td[1]/div/div/div/p[4]/span[2][1]')
todayY=browser.find_elements_by_xpath('//*[@id="app"]/section/main/div/div[4]/div/div/div[3]/div[2]/table/tbody/tr/td[2]/div/div/div/p[1]/span[2][1]')
todayR=browser.find_elements_by_xpath('//*[@id="app"]/section/main/div/div[4]/div/div/div[3]/div[2]/table/tbody/tr/td[2]/div/div/div/p[2]/span[2][1]')
MY=browser.find_elements_by_xpath('//*[@id="app"]/section/main/div/div[4]/div/div/div[3]/div[2]/table/tbody/tr/td[2]/div/div/div/p[3]/span[2][1]')
MR=browser.find_elements_by_xpath('//*[@id="app"]/section/main/div/div[4]/div/div/div[3]/div[2]/table/tbody/tr/td[2]/div/div/div/p[4]/span[2][1]')
totalY=browser.find_elements_by_xpath('//*[@id="app"]/section/main/div/div[4]/div/div/div[3]/div[2]/table/tbody/tr/td[3]/div/div/div/p[1]/span[2][1]')
totalR=browser.find_elements_by_xpath('//*[@id="app"]/section/main/div/div[4]/div/div/div[3]/div[2]/table/tbody/tr/td[3]/div/div/div/p[2]/span[2][1]')
DUT=browser.find_elements_by_xpath('//*[@id="app"]/section/main/div/div[4]/div/div/div[3]/div[2]/table/tbody/tr/td[3]/div/div/div/p[3]/span[2][1]')

RTP = RTP[0].text
siteStatus= siteStatus[0].text
PVC= PVC[0].text
SCT= SCT[0].text
todayY= todayY[0].text
todayR= todayR[0].text
MY= MY[0].text
MR= MR[0].text
totalY= totalY[0].text
totalR= totalR[0].text
DUT= DUT[0].text

# create temporary list
temporaryData= {'Real-time Power(kW)':RTP,
                   'Site Status': siteStatus,
                   'PV Capacity(kWp)': PVC,
                   'Site Creation Time': SCT,
                   'Today Yield(kWh)': todayY,
                   'Today Revenue(R$)': todayR,
                   'Monthly Yield(kWh)': MY,
                   'Monthly Revenue(R$)': MR,
                   'Total Yield(kWh)': totalY,
                   'Total Revenue(R$)': totalR,
                   'Data Update Time': DUT}

dataResult.append(temporaryData,ignore_index=True)

# create df
totalData=totalData.append(dataResult)

# close browser
browser.close()

print("Dados obtidos da plataforma Livoltek")


Dados obtidos da plataforma Livoltek


# Create path and save/update json file

In [6]:
# create a new path
try:
    os.mkdir(r"C:\Users\jheff\Desktop\Json Files")
    messageNewFolders = ("Arquivo json foi criado")
except FileExistsError as e:
    messageNewFolders = ("Arquivo json foi atualizado")

 
 # if already have a json file, append a new item in json array
if messageNewFolders == "Arquivo json foi atualizado":
    def write_json(new_data, filename='data.json'):
        with open(r"C:\Users\jheff\Desktop\Json Files\file.json",'r+') as file:
            file_data = json.load(file)
            file_data.append(new_data)
            file.seek(0)
            json.dump(file_data, file, indent = 4)
        
    y = temporaryData

    write_json(y)

# else create a new json array file
else:
    newJsonFile = [temporaryData]
    with open(r"C:\Users\jheff\Desktop\Json Files\file.json","w+") as f:
        json.dump(newJsonFile, f)
        
print(messageNewFolders)

Arquivo json foi atualizado


# Create dataframe to update g.sheet dashboard

In [7]:
# List 
datalist = [ ('Real-time Power(kW)',RTP,'Today Yield(kWh)', todayY,'Total Yield(kWh)', totalY)]
#Create a DataFrame object
dfsheet = pd.DataFrame(datalist, columns = ['col1' , 'col2', 'col3','col4' , 'col5', 'col6'])
#Add new ROWS
dfsheet.loc[1]=[ 'Site Status', siteStatus,'Today Revenue(R$)', todayR,'Total Revenue(R$)',totalR]
dfsheet.loc[2]=[ 'PV Capacity(kWp)', PVC,'Monthly Yield(kWh)', MY,'Data Update Time', DUT]
dfsheet.loc[3]=[ 'Site Creation Time', SCT,'Monthly Revenue(R$)', MR,"-","-"]

print("Dados prontos para serem atualizados no Dashboard")

,col1,col2,col3,col4,col5,col6
0,Real-time Power(kW),0,Today Yield(kWh),0,Total Yield(kWh),335.79
1,Site Status,All Offline,Today Revenue(R$),0,Total Revenue(R$),0
2,PV Capacity(kWp),6,Monthly Yield(kWh),306.72,Data Update Time,2021-10-10 17:43:20
3,Site Creation Time,2021-09-20 17:37:20,Monthly Revenue(R$),0,-,-


# Update g.sheet dashboard

In [8]:
# set spreadsheet update
serviceAccountFile = 'keyscrap.json'
scopes = ['https://www.googleapis.com/auth/spreadsheets']

creds = None
creds = service_account.Credentials.from_service_account_file(
        serviceAccountFile, scopes=scopes)

# The ID spreadsheet.
spreadsheetID = '1H5e3g8XDeKXpIZ6Bo4dv0rC18Uqr3b_Ve_H1bMJN1ss'   

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()
sheetUpdate = sheet.values().get(spreadsheetId=spreadsheetID, range="Dados atualizados!A1").execute()

df = dfsheet

valueRangeBody = {"values":df.to_numpy().tolist(),"majorDimension": "DIMENSION_UNSPECIFIED"}

requestUpdate = sheet.values().update(spreadsheetId=spreadsheetID, 
                                range="Dados atualizados!D8", 
                                valueInputOption="USER_ENTERED", 
                                body=valueRangeBody)

# execute update
response = requestUpdate.execute()

print("Dashboard atualizado")

print("A automação foi finalizada com sucesso!")


Dashboard atualizado
A automação foi finalizada com sucesso!
